In [16]:
!pip install streamlit pyngrok pandas plotly


In [17]:
%%writefile pipeline.py
import pandas as pd

def preprocess(df):
    # normalize column names
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]
    df = df.dropna()
    return df

def feature_engineering(df):
    if 'heart_rate' in df.columns:
        df['hr_rolling_mean'] = df['heart_rate'].rolling(5, min_periods=1).mean()
    return df

def model_prediction(df):
    if 'heart_rate' in df.columns:
        df['stress_score'] = (df['heart_rate'] - df['heart_rate'].mean()) / df['heart_rate'].std()
    return df

def detect_anomalies(df):
    # Heart anomaly
    if 'stress_score' in df.columns:
        df['heart_anomaly'] = df['stress_score'].abs() > 2
    else:
        df['heart_anomaly'] = False

    # Sleep anomaly
    sleep_col = next((c for c in ['hours_sleep','sleep_hours','sleep'] if c in df.columns), None)
    if sleep_col:
        df['sleep_anomaly'] = (df[sleep_col] < 4) | (df[sleep_col] > 10)
    else:
        df['sleep_anomaly'] = False

    # Steps anomaly
    step_col = next((c for c in ['steps','step_count','walk_steps','activity_steps'] if c in df.columns), None)
    if step_col:
        df['steps_anomaly'] = df[step_col] < 3000
    else:
        df['steps_anomaly'] = False

    # Unified anomaly flag
    df['is_anomaly'] = df[['heart_anomaly', 'sleep_anomaly', 'steps_anomaly']].any(axis=1)
    return df


Overwriting pipeline.py


In [18]:
%%writefile app.py
import streamlit as st
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
from pipeline import preprocess, feature_engineering, model_prediction, detect_anomalies

st.set_page_config(layout="wide")
st.title("🏥 Health • Heart • Sleep • Steps — Anomaly Dashboard")

upload = st.file_uploader("📁 Upload CSV or JSON", type=["csv","json"])

if upload:
    df = pd.read_csv(upload) if upload.name.endswith(".csv") else pd.DataFrame(json.load(upload))

    st.subheader("📋 Raw Data Sample")
    st.dataframe(df.head())

    if st.button("🚀 Run Pipeline"):
        df = preprocess(df)
        df = feature_engineering(df)
        df = model_prediction(df)
        df = detect_anomalies(df)

        st.success("✔ Pipeline Executed!")

        # Detect timestamp column or create one
        date_col = next((c for c in df.columns if 'date' in c or 'time' in c), None)
        if date_col:
            df['timestamp'] = pd.to_datetime(df[date_col])
        else:
            df['timestamp'] = pd.to_datetime(df.index)

        # Date filtering
        min_d, max_d = df['timestamp'].min().date(), df['timestamp'].max().date()
        start, end = st.date_input("📅 Date Range", (min_d, max_d))
        df = df[(df['timestamp'].dt.date >= start) & (df['timestamp'].dt.date <= end)]

        st.subheader("📊 Visual Insights")

        # ==========================
        # HEART RATE VISUALIZATION
        # ==========================
        if 'heart_rate' in df.columns:
            st.write("❤️ Heart Rate with Anomalies")

            if 'heart_anomaly' not in df.columns:
                df['heart_anomaly'] = False

            hr_anom = df[df['heart_anomaly'] == True]

            fig = px.line(df, x='timestamp', y='heart_rate', title="Heart Rate")
            fig.add_trace(go.Scatter(
                x=hr_anom['timestamp'],
                y=hr_anom['heart_rate'],
                mode='markers',
                marker=dict(color='red', size=8),
                name='Heart Anomaly'
            ))
            st.plotly_chart(fig, use_container_width=True)

        # ==========================
        # SLEEP VISUALIZATION
        # ==========================
        sleep_col = next((c for c in ['hours_sleep','sleep_hours','sleep'] if c in df.columns), None)
        if sleep_col:
            st.write("😴 Sleep Duration & Abnormal Patterns")

            if 'sleep_anomaly' not in df.columns:
                df['sleep_anomaly'] = False

            sl_anom = df[df['sleep_anomaly'] == True]

            fig_sleep = px.line(df, x='timestamp', y=sleep_col, title="Sleep Duration")
            fig_sleep.add_trace(go.Scatter(
                x=sl_anom['timestamp'],
                y=sl_anom[sleep_col],
                mode='markers',
                marker=dict(color='orange', size=8),
                name='Sleep Anomaly'
            ))
            st.plotly_chart(fig_sleep, use_container_width=True)

        # ==========================
        # STEP COUNT BEHAVIOR & ALERTS
        # ==========================
        step_col = next((c for c in ['steps','step_count','daily_steps','activity_steps'] if c in df.columns), None)
        if step_col:
            st.subheader("🚶 Step Count Behavior & Low Activity Alerts")

            sedentary_threshold = df[step_col].quantile(0.1)
            df['low_activity'] = df[step_col] < sedentary_threshold
            low_df = df[df['low_activity'] == True]

            st.write(f"**Sedentary Threshold:** {int(sedentary_threshold)} steps")
            st.write(f"**Low Activity Days:** {len(low_df)}")

            fig_steps = px.line(df, x='timestamp', y=step_col, title="Step Count Behavior")
            fig_steps.add_trace(go.Scatter(
                x=low_df['timestamp'],
                y=low_df[step_col],
                mode='markers',
                marker=dict(color='blue', size=9),
                name='Low Activity Alert'
            ))
            st.plotly_chart(fig_steps, use_container_width=True)

        else:
            st.warning("⚠ No step count column found. Supported names: steps, step_count, walk_steps, activity_steps")

        # ==========================
        # REPORT GENERATION
        # ==========================
        st.subheader("📄 Anomaly Summary Report")
        report = df[df['is_anomaly'] == True]

        if report.empty:
            st.info("No anomalies detected in this range.")
        else:
            st.dataframe(report)
            csv = report.to_csv(index=False).encode('utf-8')
            st.download_button("⬇ Download CSV Report", csv, "anomaly_report.csv", "text/csv")

else:
    st.info("Please upload your fitness dataset to begin.")


Overwriting app.py


In [19]:
!streamlit run app.py --server.port 8501 &>/dev/null &


In [20]:
from pyngrok import ngrok
ngrok.set_auth_token("383tQvapeRmx3AaadCPii6JwViS_71qvd3PB94JXrbaC26nvc")
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://laurice-ventricular-raylene.ngrok-free.dev" -> "http://localhost:8501">

In [21]:

from pyngrok import ngrok

ngrok.kill()  # to kill old tunnels if any
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://laurice-ventricular-raylene.ngrok-free.dev" -> "http://localhost:8501">